# Profitable app profiles for the App Store and Google Play markets
Our company builds free apps for the App Store and Google Play markets. Its primary source of revenue consists of in-app ads, which are mostly influenced by the number of users. This study aims to find mobile app profiles that are profitable and to enable the team of developers to make data-driven decisions.

## Import modules

In [1]:
import itertools
from csv import reader

## Data sets used
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. Two simplify our study, the following subsets of data are used:
 * Mobile App Store - provided by Ramanathan through Kaggle ([link](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps))
 * Google Play Store Apps - provided by Lavanya Gupta through Kaggle ([link](https://www.kaggle.com/lava18/google-play-store-apps))

## Data preview
Functions used in this section:

In [2]:
# Open and preview the data sets
def open_data(file_name):
    '''Opens the data file and returns a list of lists.'''

    opened_file = open(file_name, encoding='utf-8')

    read_file = reader(opened_file)

    data = list(read_file)

    return data[0], data[1:]

def explore_data(dataset, start=0, end=5, rows_and_columns=True):
    '''Prints a slice of the specified data set and the number of rows and columns.'''

    dataset_slice = dataset[start:end]
    
    for row in dataset_slice:

        print(row, '\n')

    if rows_and_columns:
        
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
# App Store data set
ios_header, ios_apps = open_data('AppleStore.csv')

print(ios_header, '\n')
explore_data(ios_apps)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'] 

['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1'] 

Number of rows: 7197
Number of columns: 16


The App Store data set has 7197 apps and 16 columns. The columns that might be useful for the purpose of our analysis are 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

In [4]:
# Google Play data set
android_header, android_apps = open_data('googleplaystore.csv')

print(android_header, '\n')
explore_data(android_apps)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Eve

The Google Play data set has 10841 apps and 13 columns. The columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

## Data cleaning
### Remove wrong data
According to the [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) of the android data set, the data of row 10472 is wrong. This row has a missing value in the 'Category' column and will be removed.

In [5]:
del android_apps[10472]

### Remove repeated data
We will check if the apps have repeated entries in each data set and only keep an entry for each app based on a selected criterion. For the App Store data set, the criteria used is the number of user ratings, and for the Google Play data set, the criteria used is the number of reviews. The purpose of these criteria is to select the most recent entry.

In [6]:
def remove_repeated(dataset, id_col, count_col):
    '''Removes repeated apps from the data set.'''

    apps_count = {}

    # Get the highest count for each app
    for idx, app in enumerate(dataset):

        app_id = app[id_col]
        count = int(app[count_col])

        if app_id in apps_count:

            if apps_count[app_id][1] < count: apps_count[app_id] = (idx, count)
            
        else:
            apps_count[app_id] = (idx, count)

    # Remove repeated entries
    keep = [False] * len(dataset)

    for app in apps_count: keep[apps_count[app][0]] = True

    removed = [app[1] for app in list(itertools.compress(dataset, [not i for i in keep]))]
    
    dataset = list(itertools.compress(dataset, keep))
    
    print('Number of entries removed:', len(removed))
    print('Number of entries kept:', len(dataset))
    
    return dataset, removed

In [7]:
ios_apps, removed = remove_repeated(ios_apps, 0, 5)

Number of entries removed: 0
Number of entries kept: 7197


In [8]:
android_apps, removed = remove_repeated(android_apps, 0, 3)

Number of entries removed: 1181
Number of entries kept: 9659


### Remove non-English apps
We are only interested in keeping apps that are directed toward an English-speaking audience. English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.). All these characters that are specific to English text are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it. However, some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters.

In [9]:
def remove_non_english(dataset, name_col):
    '''Removes non-English apps from the data set.'''

    keep = [sum(ord(char) > 127 for char in app[name_col]) <= 3 for app in dataset]

    removed = [app[name_col] for app in list(itertools.compress(dataset, [not i for i in keep]))]
    
    dataset = list(itertools.compress(dataset, keep))
    
    print('Number of entries removed:', len(removed))
    print('Number of entries kept:', len(dataset))

    return dataset, removed

In [10]:
ios_apps, removed = remove_non_english(ios_apps, 1)
#print(removed)

Number of entries removed: 1014
Number of entries kept: 6183


In [11]:
android_apps, removed = remove_non_english(android_apps, 0)
#print(removed)

Number of entries removed: 45
Number of entries kept: 9614


### Remove paid apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our primary source of revenue consists of in-app ads. Below, we remove the paid apps from both data sets.

In [12]:
def remove_paid(dataset, price_col):
    '''Removes paid apps from the data set.'''

    keep = [app[price_col] == '0' or app[price_col] == '0.0' for app in dataset]

    length = len(dataset)
    
    dataset = list(itertools.compress(dataset, keep))
    
    print('Number of entries removed:', length - len(dataset))
    print('Number of entries kept:', len(dataset))

    return dataset

In [13]:
ios_apps = remove_paid(ios_apps, 4)

Number of entries removed: 2961
Number of entries kept: 3222


In [14]:
android_apps = remove_paid(android_apps, 7)

Number of entries removed: 750
Number of entries kept: 8864


## Data analysis

### Most common apps by genre
As we mentioned in the introduction, we aim to determine the kinds of apps that are likely to attract more users. To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play;
2. If the app has a good response from users, we then develop it further;
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful in both markets. We will begin by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

We'll build two functions we can use to analyze the frequency tables:
 * One function to generate frequency tables that show percentages
 * Another function that we can use to display the percentages in a descending order

In [15]:
def freq_table(dataset, col):
    '''Generates a frequency table that shows percentages for the specified column.'''

    table = {}
    
    for app in dataset:
        
        genre = app[col]
        
        if genre in table:
            
            table[genre] += 1
        else:
            table[genre] = 1

    table = {k: v / len(dataset) * 100 for k, v in table.items()}
    
    return table

def display_table(table):
    '''Displays the values of the table in descending order.'''
    
    table = sorted(table.items(), key = lambda x: x[1], reverse = True)
    
    for genre in table: print('{}: {:.1f}'.format(genre[0], genre[1]))

We start by examining the frequency table for the `prime_genre` column of the App Store data set.

In [16]:
ios_genre_table = freq_table(ios_apps, -5)
display_table(ios_genre_table)

Games: 58.2
Entertainment: 7.9
Photo & Video: 5.0
Education: 3.7
Social Networking: 3.3
Shopping: 2.6
Utilities: 2.5
Sports: 2.1
Music: 2.0
Health & Fitness: 2.0
Productivity: 1.7
Lifestyle: 1.6
News: 1.3
Travel: 1.2
Finance: 1.1
Weather: 0.9
Food & Drink: 0.8
Reference: 0.6
Business: 0.5
Book: 0.4
Navigation: 0.2
Medical: 0.2
Catalogs: 0.1


We can see that among the free English apps, more than half (58.2%) are games. Entertainment apps are close to 8%, followed by photo and video apps (5%). Only 3.7% of the apps are designed for education, followed by social networking apps which amount for 3.3% of the apps in our data set. 

It seems that the free English apps of the App Store are mostly designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.). In contrast, apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are rarer. However, the fact that fun apps are the most numerous doesn't imply that they have the most significant number of users. The demand might not be the same as the offer. 

Let's continue by examining the `Genres` and `Category` columns of the Google Play data set (two columns which seem to be related).

In [17]:
android_category_table = freq_table(android_apps, 1)
display_table(android_category_table)

FAMILY: 18.9
GAME: 9.7
TOOLS: 8.5
BUSINESS: 4.6
LIFESTYLE: 3.9
PRODUCTIVITY: 3.9
FINANCE: 3.7
MEDICAL: 3.5
SPORTS: 3.4
PERSONALIZATION: 3.3
COMMUNICATION: 3.2
HEALTH_AND_FITNESS: 3.1
PHOTOGRAPHY: 2.9
NEWS_AND_MAGAZINES: 2.8
SOCIAL: 2.7
TRAVEL_AND_LOCAL: 2.3
SHOPPING: 2.2
BOOKS_AND_REFERENCE: 2.1
DATING: 1.9
VIDEO_PLAYERS: 1.8
MAPS_AND_NAVIGATION: 1.4
FOOD_AND_DRINK: 1.2
EDUCATION: 1.2
ENTERTAINMENT: 1.0
LIBRARIES_AND_DEMO: 0.9
AUTO_AND_VEHICLES: 0.9
HOUSE_AND_HOME: 0.8
WEATHER: 0.8
EVENTS: 0.7
PARENTING: 0.7
ART_AND_DESIGN: 0.6
COMICS: 0.6
BEAUTY: 0.6


The landscape seems different on Google Play, since there are not that many apps designed for fun. A significant number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

[![img](https://s3.amazonaws.com/dq-content/350/py1m8_family.png)](https://play.google.com/store/apps/category/FAMILY?hl=en)

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the `Genres` column:

In [18]:
android_genre_table = freq_table(android_apps, -4)
display_table(android_genre_table)

Tools: 8.4
Entertainment: 6.1
Education: 5.3
Business: 4.6
Lifestyle: 3.9
Productivity: 3.9
Finance: 3.7
Medical: 3.5
Sports: 3.5
Personalization: 3.3
Communication: 3.2
Action: 3.1
Health & Fitness: 3.1
Photography: 2.9
News & Magazines: 2.8
Social: 2.7
Travel & Local: 2.3
Shopping: 2.2
Books & Reference: 2.1
Simulation: 2.0
Dating: 1.9
Arcade: 1.9
Video Players & Editors: 1.8
Casual: 1.8
Maps & Navigation: 1.4
Food & Drink: 1.2
Puzzle: 1.1
Racing: 1.0
Libraries & Demo: 0.9
Role Playing: 0.9
Auto & Vehicles: 0.9
Strategy: 0.9
House & Home: 0.8
Weather: 0.8
Events: 0.7
Adventure: 0.7
Comics: 0.6
Art & Design: 0.6
Beauty: 0.6
Parenting: 0.5
Card: 0.5
Casino: 0.4
Trivia: 0.4
Educational;Education: 0.4
Board: 0.4
Educational: 0.4
Education;Education: 0.3
Word: 0.3
Casual;Pretend Play: 0.2
Music: 0.2
Entertainment;Music & Video: 0.2
Puzzle;Brain Games: 0.2
Racing;Action & Adventure: 0.2
Casual;Brain Games: 0.1
Casual;Action & Adventure: 0.1
Arcade;Action & Adventure: 0.1
Action;Action & Ad

The difference between the `Genres` and the `Category` columns is not crystal clear. One thing we can notice is that the `Genres` column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the `Category` column moving forward.

So far, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

### Most popular apps by genre

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` app.

In [19]:
def average_table(dataset, group_col, avg_col):

    groups_avg = {}
    
    for app in dataset:
        
        group = app[group_col]
        val = float(app[avg_col])
       
        if group in groups_avg:
            
            aux = groups_avg[group]
            groups_avg[group] = (aux[0] + val, aux[1] + 1)
        else:
            groups_avg[group] = (val, 1)

    groups_avg = {k: v[0] / v[1] for k, v in groups_avg.items()}
    
    return groups_avg

In [20]:
ios_ratings_avg = average_table(ios_apps, -5, 5)
display_table(ios_ratings_avg)

Navigation: 86090.3
Reference: 74942.1
Social Networking: 71548.3
Music: 57326.5
Weather: 52279.9
Book: 39758.5
Food & Drink: 33333.9
Finance: 31467.9
Photo & Video: 28441.5
Travel: 28243.8
Shopping: 26919.7
Health & Fitness: 23298.0
Sports: 23008.9
Games: 22788.7
News: 21248.0
Productivity: 21028.4
Utilities: 18684.5
Lifestyle: 16485.8
Entertainment: 14029.8
Business: 7491.1
Education: 7004.0
Catalogs: 4004.0
Medical: 612.0


Navigation apps have the highest number of average user reviews. However, this figure is heavily influenced by Waze and Google Maps, which together have close to half a million user reviews.

In [21]:
for app in ios_apps:

    if app[-5] == 'Navigation': print('{}: {}'.format(app[1], app[5]))

Waze - GPS Navigation, Maps & Real-time Traffic: 345046
Google Maps - Navigation & Transit: 154911
Geocaching®: 12811
CoPilot GPS – Car Navigation & Offline Maps: 3582
ImmobilienScout24: Real Estate Search in Germany: 187
Railway Route Search: 5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify and Shazam heavily influence the average number. Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating.

In [22]:
for app in ios_apps:

    if app[-5] == 'Reference': print('{}: {}'.format(app[1], app[5]))

Bible: 985920
Dictionary.com Dictionary & Thesaurus: 200047
Dictionary.com Dictionary & Thesaurus for iPad: 54175
Google Translate: 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran: 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition: 17588
Merriam-Webster Dictionary: 16849
Night Sky: 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE): 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools: 4693
GUNS MODS for Minecraft PC Edition - Mods Tools: 1497
Guides for Pokémon GO - Pokemon GO News and Cheats: 826
WWDC: 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free: 718
VPN Express: 14
Real Bike Traffic Rider Virtual Reality Glasses: 8
教えて!goo: 0
Jishokun-Japanese English Dictionary & Translator: 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

For the Google Play market, we have data about the number of installs, so we should be able to get a clearer picture of genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [23]:
android_installs_table = freq_table(android_apps, 5)
display_table(android_installs_table)

1,000,000+: 15.7
100,000+: 11.6
10,000,000+: 10.5
10,000+: 10.2
1,000+: 8.4
100+: 6.9
5,000,000+: 6.8
500,000+: 5.6
50,000+: 4.8
5,000+: 4.5
10+: 3.5
500+: 3.2
50,000,000+: 2.3
100,000,000+: 2.1
50+: 1.9
5+: 0.8
1+: 0.5
500,000,000+: 0.3
1,000,000,000+: 0.2
0+: 0.0
0: 0.0


Although this data is not very precise, it suffices for getting an idea which app genres attract the most users. We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

In [24]:
for app in android_apps:

    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    
    app[5] = float(n_installs)
    
android_ratings_avg = average_table(android_apps, 1, 5)
display_table(android_ratings_avg)

COMMUNICATION: 38456119.2
VIDEO_PLAYERS: 24727872.5
SOCIAL: 23253652.1
PHOTOGRAPHY: 17840110.4
PRODUCTIVITY: 16787331.3
GAME: 15588015.6
TRAVEL_AND_LOCAL: 13984077.7
ENTERTAINMENT: 11640705.9
TOOLS: 10801391.3
NEWS_AND_MAGAZINES: 9549178.5
BOOKS_AND_REFERENCE: 8767811.9
SHOPPING: 7036877.3
PERSONALIZATION: 5201482.6
WEATHER: 5074486.2
HEALTH_AND_FITNESS: 4188822.0
MAPS_AND_NAVIGATION: 4056941.8
FAMILY: 3695641.8
SPORTS: 3638640.1
ART_AND_DESIGN: 1986335.1
FOOD_AND_DRINK: 1924897.7
EDUCATION: 1833495.1
BUSINESS: 1712290.1
LIFESTYLE: 1437816.3
FINANCE: 1387692.5
HOUSE_AND_HOME: 1331540.6
DATING: 854028.8
COMICS: 817657.3
AUTO_AND_VEHICLES: 647317.8
LIBRARIES_AND_DEMO: 638503.7
PARENTING: 542603.6
BEAUTY: 513151.9
EVENTS: 253542.2
MEDICAL: 120550.6


Communication apps have the most average installs (38,456,119). This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts).

In [25]:
for app in android_apps:

    if app[1] == 'COMMUNICATION' and app[5] >= 1E9: print('{}: {}'.format(app[0], app[5]))

WhatsApp Messenger: 1000000000.0
Messenger – Text and Video Chat for Free: 1000000000.0
Skype - free IM & video calls: 1000000000.0
Google Chrome: Fast & Secure: 1000000000.0
Gmail: 1000000000.0
Hangouts: 1000000000.0


This pattern is repeated for video players apps (Youtube, Google Play Movies & TV, or MX Player), social apps (Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors) or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against. The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible. The books and reference genre looks reasonably popular, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

In [26]:
for app in android_apps:

    if app[1] == 'BOOKS_AND_REFERENCE': print('{}: {}'.format(app[0], app[5]))

E-Book Read - Read Book for free: 50000.0
Download free book with green book: 100000.0
Wikipedia: 10000000.0
Cool Reader: 10000000.0
Free Panda Radio Music: 100000.0
Book store: 1000000.0
FBReader: Favorite Book Reader: 10000000.0
English Grammar Complete Handbook: 500000.0
Free Books - Spirit Fanfiction and Stories: 1000000.0
Google Play Books: 1000000000.0
AlReader -any text book reader: 5000000.0
Offline English Dictionary: 100000.0
Offline: English to Tagalog Dictionary: 500000.0
FamilySearch Tree: 1000000.0
Cloud of Books: 1000000.0
Recipes of Prophetic Medicine for free: 500000.0
ReadEra – free ebook reader: 1000000.0
Anonymous caller detection: 10000.0
Ebook Reader: 5000000.0
Litnet - E-books: 100000.0
Read books online: 5000000.0
English to Urdu Dictionary: 500000.0
eBoox: book reader fb2 epub zip: 1000000.0
English Persian Dictionary: 500000.0
Flybook: 500000.0
All Maths Formulas: 1000000.0
Ancestry: 5000000.0
HTC Help: 10000000.0
English translation from Bengali: 100000.0
Pdf

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [27]:
for app in android_apps:

    if app[1] == 'BOOKS_AND_REFERENCE' and app[5] >= 100E6: print('{}: {}'.format(app[0], app[5]))

Google Play Books: 1000000000.0
Bible: 100000000.0
Amazon Kindle: 100000000.0
Wattpad 📖 Free Books: 100000000.0
Audiobooks from Audible: 100000000.0


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [28]:
for app in android_apps:

    if app[1] == 'BOOKS_AND_REFERENCE' and app[5] >= 1E6 and app[5] <= 100E6: print('{}: {}'.format(app[0], app[5]))

Wikipedia: 10000000.0
Cool Reader: 10000000.0
Book store: 1000000.0
FBReader: Favorite Book Reader: 10000000.0
Free Books - Spirit Fanfiction and Stories: 1000000.0
AlReader -any text book reader: 5000000.0
FamilySearch Tree: 1000000.0
Cloud of Books: 1000000.0
ReadEra – free ebook reader: 1000000.0
Ebook Reader: 5000000.0
Read books online: 5000000.0
eBoox: book reader fb2 epub zip: 1000000.0
All Maths Formulas: 1000000.0
Ancestry: 5000000.0
HTC Help: 10000000.0
Moon+ Reader: 10000000.0
English-Myanmar Dictionary: 1000000.0
Golden Dictionary (EN-AR): 1000000.0
All Language Translator Free: 1000000.0
Bible: 100000000.0
Amazon Kindle: 100000000.0
Aldiko Book Reader: 10000000.0
Wattpad 📖 Free Books: 100000000.0
Dictionary - WordWeb: 5000000.0
50000 Free eBooks & Free AudioBooks: 5000000.0
Al-Quran (Free): 10000000.0
Al Quran Indonesia: 10000000.0
Al'Quran Bahasa Indonesia: 10000000.0
Al Quran Al karim: 1000000.0
Al Quran : EAlim - Translations & MP3 Offline: 5000000.0
Koran Read &MP3 30 

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries. Hence, it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.


## Conclusions
This project analyzed data about the apps for the App Store and Google Play markets to recommend an app profile that can be profitable for both markets. We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be a good idea. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.